In [ ]:
#default_exp data.training_data

In [ ]:
!nbdev_build_lib

Converted 00_jobs.ipynb.
Converted 01-create-sample-data.ipynb.
Converted 02-preprocess.ipynb.
Converted 03-feature-calc.ipynb.
Converted 04-training-data.ipynb.
Converted 05-train-model.ipynb.
Converted index.ipynb.


In [ ]:
#export
from typing import Dict
from datetime import datetime, timezone, timedelta
import random
import math
import dask.dataframe as dd
import numpy as np

from hopeit.app.context import EventContext
from hopeit.app.events import Spawn, SHUFFLE
from hopeit.app.api import event_api
from hopeit.app.logger import app_logger

from fraud_poc.jobs import get_client, FeatureCalcJob, TrainingDataJob

In [ ]:
#export
__steps__ = ['run']

logger = app_logger()

In [ ]:
#export
def _merge_feature_datasets(datasets: Dict[str, str]):
    df = None
    for key, path in datasets.items():
        df_key = dd.read_parquet(path, engine='fastparquet')
        if df is not None:
            df = df.merge(df_key, left_on='order_id', right_on='order_id', suffixes=('', '_DROP'))
            keep_cols = [c for c in df.columns if c[:-5] != '_DROP']
            df = df[keep_cols]
        else:
            df = df_key
    return df


def _add_labels(df):
    df['is_fraud'] = ((df.emails_by_customer_id + df.ip_addrs_by_customer_id) > 12) | \
            ((df.order_amount > 1.1 * df.order_amount_mean_by_email) & (df.emails_by_customer_id > 5))
    df['is_fraud'] = df['is_fraud'].apply(lambda x: int(x & (random.random() > 0.5)), meta=('is_fraud', int))
    return df

def _add_sample_flag(df, subsample_not_fraud: float):
    df['sample'] = df['is_fraud'].apply(lambda x: int((x > 0) | (random.random() > (1.-subsample_not_fraud))), meta=('sample', int))
    return df

def _add_validation_flag(df):
    now = datetime.now(tz=timezone.utc)
    now_epoch = now.timestamp()
    df['now'] = now
    df['elapsed_wgt'] = df['order_date'].apply(lambda x: math.log(max(0.001, 1. - (now_epoch - x.timestamp())/now_epoch)) + 1., meta=('elapsed_wgt', float))
    df['validation'] = df['elapsed_wgt'].apply(lambda x: int((max(0, x)  * random.random()) > 0.8), meta=('validation', int))
    return df

def _add_fold_number(df, num_folds):
    df['fold'] = df['is_fraud'].apply(lambda x: random.randint(0, num_folds), meta=('fold', int)) 
    return df

In [ ]:
#export
def run(job: FeatureCalcJob, context: EventContext) -> TrainingDataJob:
    base_path = context.env['data']['training']
    num_folds = context.env['training_data']['num_folds']
    subsample_not_fraud = context.env['training_data']['subsample_not_fraud']
    
    client = get_client(context)
    try:
        df = _merge_feature_datasets(job.features)
        df = _add_labels(df)
        df = _add_sample_flag(df, subsample_not_fraud)
        df = _add_validation_flag(df)
        df = _add_fold_number(df, num_folds)
        
        sampled_save_path = f"{base_path}/sampled/"
        logger.info(context, f"Saving sampled training dataset to {sampled_save_path}...")
        df_sample = df[df['sample'] > 0]
        df_sample = df_sample.set_index('fold')
        df_sample.to_parquet(sampled_save_path)
        
        valid_save_path = f"{base_path}/validation/"
        logger.info(context, f"Saving weighted validation dataset to {valid_save_path}...")
        df_validation = df[df['validation'] >0 ]
        df_validation.to_parquet(valid_save_path)

        return TrainingDataJob(
            sources=job.features,
            sampled=sampled_save_path,
            validation=valid_save_path
        )
    except Exception as e:
        logger.error(context, e)
        return None
    finally:
        client.close()

### Test from notebook

In [ ]:
from hopeit.testing.apps import config, execute_event

app_config = config('config/training-pipeline.json')
job = FeatureCalcJob(sources={'customer_id': './data/partitioned/customer_id/', 'email': './data/partitioned/email'}, 
                     features={'customer_id': './data/features/customer_id/', 'email': './data/features/email/'})
result = await execute_event(app_config, 'data.training-data', job)
result

2020-07-06 17:39:06,929 | INFO | fraud-poc 0.0.1 data.training-data leo-legion 32155 | Saving sampled training dataset to ./data/training/sampled/... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-06T17:39:06.313547+00:00 | stream.name= | stream.msg_id= | stream.consumer_group=
2020-07-06 17:39:08,763 | INFO | fraud-poc 0.0.1 data.training-data leo-legion 32155 | Saving weighted validation dataset to ./data/training/validation/... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-06T17:39:06.313547+00:00 | stream.name= | stream.msg_id= | stream.consumer_group=


TrainingDataJob(sources={'customer_id': './data/features/customer_id/', 'email': './data/features/email/'}, sampled='./data/training/sampled/', validation='./data/training/validation/')

In [ ]:
dd.read_parquet(result.sampled).head()

,order_id,order_date,email,ip_addr,order_amount,customer_id,emails_by_customer_id,ip_addrs_by_customer_id,order_amounts_by_customer_id,order_amount_mean_by_customer_id,...,order_amount_mean_by_email,order_amount_std_by_email,order_amount_min_by_email,order_amount_max_by_email,order_amount_sum_by_email,is_fraud,sample,now,elapsed_wgt,validation
fold,,,,,,,,,,,,,,,,,,,,,
0,e59e47bd-d811-4365-a0dd-ebc50fb168a8,2020-06-25 05:42:09+00:00,0e4e56804a60d7724cf810d0c58fab2169b34a1f,e6520b132ead6da81be53447ae2af04768f38db1,908.071566,fe72646c-059f-4b6d-9336-317302280f3b,1,1,"(908.0715663439621, 0.0, 908.0715663439621, 90...",908.071566,...,908.071566,0.000000,908.071566,908.071566,908.071566,0,1,2020-07-06 17:39:06.905718+00:00,0.999377,1
0,03f76936-9565-4b2f-80ba-8e9712095b70,2020-05-31 01:06:31+00:00,6ccf833df93f1b1f6b4de578b74974b78d09be46,b75f088436003efcb91d7ff3fe7884585bde97eb,605.226661,ddd72871-754b-484e-9880-a5c5f855c3c9,1,2,"(424.9923355918328, 180.2343250721702, 244.758...",424.992336,...,424.992336,180.234325,244.758011,605.226661,849.984671,0,1,2020-07-06 17:39:06.905718+00:00,0.998009,0
0,93773106-c34c-43bf-af85-3c4a68ca7905,2020-07-06 10:19:35+00:00,51837b758a930c08631b9a82abd612079280e4ff,064acee509a766fbd8d30aace8daa745ea1fc3c8,820.518812,d678c9a5-c244-4552-b89a-1e89668ff0ab,2,3,"(680.0545940076871, 231.12035913567237, 375.68...",680.054594,...,896.061338,75.542526,820.518812,971.603864,1792.122676,0,1,2020-07-06 17:39:06.905718+00:00,0.999983,0
0,89e0bfd4-402b-4b31-bd51-aba4270fa9a2,2020-06-17 10:29:10+00:00,3eb60bd26aef7e16afd8b83ae9eac69495017d42,5d323ece78418b2a8715601f03f3bc81842c92a6,355.194537,d5e0bbef-df09-4379-a2ee-c24f74e56ab8,1,1,"(355.1945366777379, 0.0, 355.1945366777379, 35...",355.194537,...,355.194537,0.000000,355.194537,355.194537,355.194537,0,1,2020-07-06 17:39:06.905718+00:00,0.998953,0
0,12f5a564-cd2c-47ed-827b-4faa2d7a285c,2020-06-20 01:54:17+00:00,ecb3d7d05d53ef7eee81f3c7d8d2895752b14172,790cfb7dc82020403bfdca22ea33523e1f0cede6,203.788400,b27bae5f-4359-4ea8-8ae7-ada1dc80c0f0,1,1,"(216.122923228742, 12.334522826874462, 203.788...",216.122923,...,216.122923,12.334523,203.788400,228.457446,432.245846,0,1,2020-07-06 17:39:06.905718+00:00,0.999097,0


In [ ]:
dd.read_parquet(result.validation).head()

,order_id,order_date,email,ip_addr,order_amount,customer_id,emails_by_customer_id,ip_addrs_by_customer_id,order_amounts_by_customer_id,order_amount_mean_by_customer_id,...,order_amount_std_by_email,order_amount_min_by_email,order_amount_max_by_email,order_amount_sum_by_email,is_fraud,sample,now,elapsed_wgt,validation,fold
3,a7ad0661-f86d-4388-8b5f-5c3051c39c07,2020-06-05 02:43:32+00:00,09da81d40cb2939e7a48331825837d039d2b0784,02bf0acfcbc290d72b7765fb1de00fa796ca0d88,539.827743,004c9254-4e65-47f6-88e4-d262dce434a9,1,1,"(539.8277429844641, 0.0, 539.8277429844641, 53...",539.827743,...,0.000000,539.827743,539.827743,539.827743,0,1,2020-07-06 17:39:06.905718+00:00,0.998285,1,5
4,7f37211b-d1e4-4e43-acf9-1a17abbd3072,2020-06-16 14:17:27+00:00,f599018e431a101f6ed8ef365add95a1bcce726d,eecda0b16e78628a4e5368c437a634b5ef9d6f6e,124.542277,0055a45d-ad90-48f3-a461-ac4d6716ab48,2,2,"(547.7368392614269, 423.19456234254795, 124.54...",547.736839,...,0.000000,124.542277,124.542277,124.542277,0,1,2020-07-06 17:39:06.905718+00:00,0.998908,1,1
12,5125c87f-cb91-4247-9201-f5193fbd54b2,2020-06-25 20:49:21+00:00,c2dc3fc7e0eecd19852a0e6b7ca0d767a66dc8a8,275566619ba05657dbdb51db770d3a42989a91ae,295.131826,00d12df8-e418-4b8d-9148-b36474f0f34e,1,1,"(295.13182591906616, 0.0, 295.13182591906616, ...",295.131826,...,0.000000,295.131826,295.131826,295.131826,0,0,2020-07-06 17:39:06.905718+00:00,0.999411,1,3
24,ad1c4d90-38b3-47c4-abb6-33a017a169e7,2020-06-20 17:25:01+00:00,66c3d723ee17b2cf1db18d90d6226e8a6419c607,c544fb6eaf6ca50dea565fb1e134ac9ac3483e97,258.153994,023e9adf-5709-4446-bffd-5993504df092,3,3,"(478.0973519971869, 139.41141103989563, 258.15...",478.097352,...,190.101449,258.153994,638.356891,896.510885,0,1,2020-07-06 17:39:06.905718+00:00,0.999132,1,6
25,772d888f-692f-49b0-99bf-67c99e1cbd71,2020-06-13 13:41:05+00:00,66c3d723ee17b2cf1db18d90d6226e8a6419c607,c544fb6eaf6ca50dea565fb1e134ac9ac3483e97,638.356891,023e9adf-5709-4446-bffd-5993504df092,2,2,"(557.7253402594326, 80.63155079803204, 477.093...",557.725340,...,0.000000,638.356891,638.356891,638.356891,0,1,2020-07-06 17:39:06.905718+00:00,0.998744,1,8
